In [1]:
import run_api

Obtained my token successfully!
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}
{'data': 2, 'error': 'task_on_executing', 'error_Description': "Task is running! You don't need to start again."}
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}


In [7]:
import utils
from ner import NER
csv = utils.csv
pd = utils.pd
np = utils.np
nltk = utils.nltk
itertools = utils.itertools
re = utils.re

In [8]:
Nfiles_df_list, first_extract, second_extract = run_api.Nfiles_df_list, run_api.first_extract_df, run_api.second_extract_df

In [9]:
xls = pd.ExcelFile('strings stems.xlsx')

first = pd.read_excel(xls, 'first').dropna(axis=1, how='all').dropna(axis=0, how='all')
parent_second = pd.read_excel(xls, 'parent second').dropna(axis=1, how='all').dropna(axis=0, how='all')
bad_keywords= pd.read_excel(xls, 'bad keywords').dropna(axis=1, how='all').dropna(axis=0, how='all')

tutoring = pd.read_excel(xls, 'Tutoring').dropna(axis=1, how='all').dropna(axis=0, how='all')
payments = pd.read_excel(xls, 'payments').dropna(axis=1, how='all').dropna(axis=0, how='all')
contract = pd.read_excel(xls, 'contract').dropna(axis=1, how='all').dropna(axis=0, how='all')

utils.pre_process_df_keywords(first)
utils.pre_process_df_keywords(parent_second)
utils.pre_process_df_keywords(bad_keywords)

print('x' in parent_second["sound effects"].values)
contract

False


,name
0,contract
1,NDA


In [14]:
raw_message = """My name is Jack, the designer of Bellorita handbags (bellorita.com). We are working on a 90-second video ad on facebook. We need a video editor to help us edit the video with text, image, and footage overlay to go with the script and spokewoman.

Right now the script (225 words) is ready, but the spokewoman video is not yet, and we need your suggestion on how we need her to performe to smooth your video editing.

If you are interested, please propose with your price.

Please note that this is an easy project, and we want the lowest rate possible. Thank you.

I will pay through cash app and I need to sign an NDA

Sincerely
Jack
""".lower()
message = utils.pre_process(raw_message)

print("Breaking_2") if utils.step_1(raw_message, utils.get_two_word(message), bad_keywords) else False
print(utils.step_1(raw_message, utils.get_two_word(message), bad_keywords))


utils.find_workers(raw_message, message, parent_second), utils.find_workers(raw_message, message, first)

[]


((array(['photoshop', 'video editing'], dtype=object), 'video editing'),
 (array(['a graphic designer', 'someone', 'a video editor'], dtype=object),
  'a video editor'))

In [15]:
extractor = NER()
def extract_pay_info(second_extract, extractor):
    extracted_pay = pd.DataFrame({"field": second_extract['field3'].values, 
              "Res": list(map(extractor.extract_entities_dict,(map(utils.append_sapces,second_extract['field3'].values)))),
              "Decesion": [" "] * len(second_extract['field3'].values)
             })
    for indx, item in enumerate(extracted_pay["Res"]):
        if len(item) != '' and ("money" in item.keys() or 'cardinal' in item.keys()):
            temp = re.findall(r'\d+', str(item.values()).replace(',',""))
            try:
                extracted_pay.iloc[indx]['Decesion'] = min(list(map(int, temp)))
            except:
                extracted_pay.iloc[indx]['Decesion'] = -1
        else:
            extracted_pay.iloc[indx]['Decesion'] = -1

    return extracted_pay

In [18]:
extracted_pay = extract_pay_info(second_extract, extractor)

,index,field1,field2,field3
0,0,Boudoir Model Needed,QR Code Link to This Post\n \n ...,compensation: 50/hr or $200
1,1,Paying $5 to test a finance app,QR Code Link to This Post\n \n ...,compensation: $5
2,2,Model wanted for womenswear photoshoot/lookbook,QR Code Link to This Post\n \n ...,"compensation: $40/hr, for 4-5 hrs"
3,3,Model Needed 30-50 Female,QR Code Link to This Post\n \n ...,compensation: 250
4,4,Seeking Actresses and Models 18+,QR Code Link to This Post\n \n ...,compensation: up to 500 dollars a day
...,...,...,...,...
214,214,$150: Looking for Female Model to Visit N*de B...,QR Code Link to This Post\n \n ...,compensation: $150 cash
215,215,Need help editing a short video,QR Code Link to This Post\n \n ...,compensation: Bid
216,216,Festival Life- Van-Life Adventure Models,QR Code Link to This Post\n \n ...,compensation: $$ Weekly and Split of all Chann...
217,217,Celebrity Music Video,QR Code Link to This Post\n \n ...,compensation: Exposure


In [26]:
second_extract_pay_appended = second_extract.reset_index().merge(extracted_pay.reset_index(), on="index",how="right").drop(["index","field","Res"],axis=1)
second_extract_pay_appended.rename(columns={"field1": "title", "field2":"body", "field3": "compensation", "Decesion":"pay_amount"}, inplace=True)

In [33]:
second_extract_pay_appended

,title,body,compensation,pay_amount
0,Boudoir Model Needed,QR Code Link to This Post\n \n ...,compensation: 50/hr or $200,50
1,Paying $5 to test a finance app,QR Code Link to This Post\n \n ...,compensation: $5,5
2,Model wanted for womenswear photoshoot/lookbook,QR Code Link to This Post\n \n ...,"compensation: $40/hr, for 4-5 hrs",4
3,Model Needed 30-50 Female,QR Code Link to This Post\n \n ...,compensation: 250,250
4,Seeking Actresses and Models 18+,QR Code Link to This Post\n \n ...,compensation: up to 500 dollars a day,500
...,...,...,...,...
214,$150: Looking for Female Model to Visit N*de B...,QR Code Link to This Post\n \n ...,compensation: $150 cash,150
215,Need help editing a short video,QR Code Link to This Post\n \n ...,compensation: Bid,-1
216,Festival Life- Van-Life Adventure Models,QR Code Link to This Post\n \n ...,compensation: $$ Weekly and Split of all Chann...,-1
217,Celebrity Music Video,QR Code Link to This Post\n \n ...,compensation: Exposure,-1


In [60]:
first_extract.rename(inplace=True, columns={"field1_Text_Text": "title", "field1_Link_Link": "link"})

In [95]:
print((second_extract_pay_appended.merge(first_extract, on= "title", how="left" ).drop_duplicates().iloc[2:5]["link"].values))

['https://orangecounty.craigslist.org/tlg/d/irvine-model-wanted-for-womenswear/7061288716.html'
 'https://provo.craigslist.org/tlg/d/alpine-model-needed-female/7061291756.html'
 'https://saltlakecity.craigslist.org/tlg/d/alpine-model-needed-female/7061657275.html']


In [2]:
from fuzzywuzzy import fuzz

71

In [10]:
import pandas as pd

In [11]:
def func(x = pd.DataFrame([])):
    return x

,0
0,1
1,2
2,3
